In [2]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

## Read Data

In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

## Display sample data

In [6]:
spacex_df.describe()

,Unnamed: 0,Flight Number,class,Payload Mass (kg)
count,56.000000,56.000000,56.000000,56.000000
mean,27.500000,28.500000,0.428571,3696.648214
std,16.309506,16.309506,0.499350,2568.509679
min,0.000000,1.000000,0.000000,0.000000
25%,13.750000,14.750000,0.000000,2121.000000
50%,27.500000,28.500000,0.000000,3412.500000
75%,41.250000,42.250000,1.000000,5042.500000
max,55.000000,56.000000,1.000000,9600.000000


## Display launch sites

In [8]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [9]:
success_df= spacex_df[spacex_df['class'] == 1]
success_df_grouped = success_df.groupby('Launch Site')['class'].value_counts().reset_index()
success_df_grouped

,Launch Site,class,count
0,CCAFS LC-40,1,7
1,CCAFS SLC-40,1,3
2,KSC LC-39A,1,10
3,VAFB SLC-4E,1,4


In [18]:

max_paylaoad = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
            # TASK 1: Add a dropdown list to enable Launch Site selection
            # The default select value is for ALL sites
            # dcc.Dropdown(id='site-dropdown',...)
              dcc.Dropdown(id='site-dropdown',
                    options=[
                         {'label': 'All', 'value': 'ALL'},
                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                    ],
                    value='ALL',
                    placeholder="place holder here",
                    searchable=True
                    ),
            html.Br(),

            # TASK 2: Add a pie chart to show the total successful launches count for all sites
            # If a specific launch site was selected, show the Success vs. Failed counts for the site
            html.Div(dcc.Graph(id='success-pie-chart')),
            html.Br(),
           
            html.P("Payload range (Kg):"),
            # TASK 3: Add a slider to select payload range
            #dcc.RangeSlider(id='payload-slider',...)
            dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                marks={0: '0',
                       100: '100'},
                value=[min_payload, max_paylaoad],
                 tooltip={"placement": "bottom", "always_visible": True,
                          "template": "{value} Kg"}),
            

            # TASK 4: Add a scatter chart to show the correlation between payload and launch success
            html.Div(dcc.Graph(id='success-payload-scatter-chart')),
            ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown',component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        success_df= spacex_df[spacex_df['class'] == 1]
        success_df_grouped = success_df.groupby('Launch Site')['class'].value_counts().reset_index()
        success_df_grouped
        
        fig = px.pie(
            success_df_grouped, 
            values='count', 
            names='Launch Site', 
            title='Total Success vs Failure for All Sites'
        )
        
        return fig
    
    else:
        
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_counts = site_df['class'].value_counts().reset_index()
        fig = px.pie(
            site_counts, 
            values='count', 
            names='class', 
            title=f'Success vs Failure for {entered_site}'
        )
        return fig

        
        
        # return the outcomes piechart for a selected site
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( 
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)
def update_scatter(selected_site, payload_range):
    filtered_df = spacex_df[ (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & (spacex_df['Payload Mass (kg)'] <= payload_range[1]) ]
    if selected_site == 'ALL':
       fig = px.scatter(
            filtered_df, 
            x='Payload Mass (kg)',
            y='class',
            title='Payload Mass (kg) vs Success for All Sites',
           color="Booster Version Category"
        )
       return fig
    else:
       site_df = filtered_df[spacex_df['Launch Site']==selected_site]
       fig = px.scatter(
            site_df, 
            x='Payload Mass (kg)',
            y='class',
            title=f'Payload Mass (kg) vs Success for {selected_site}',
           color="Booster Version Category"
        )

       
       return fig



# Run the app
if __name__ == '__main__':
    app.run(mode='inline', port=8051)

#app.run(mode='inline', port=8050)